## Connect to RDS

In [1]:
import psycopg2

host = "capstone-database.cr62wyo4a7dt.us-east-2.rds.amazonaws.com"
database = "capstone"
user = "postgres"
password = "<PASSWORD_HERE>"
port = "5432"

try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connection successful")
except Exception as e:
    print(f"Error connecting to database: {e}")

Connection successful


## Read data as pandas dataframe

In [2]:
import pandas as pd

# Read data from the 'event' table
try:
    query = "SELECT * FROM event LIMIT 10;"  # Adjust the query as needed
    df = pd.read_sql(query, connection)
    print(df)
except Exception as e:
    print(f"Error reading data from table: {e}")

  eventid enrollmentid personalid   eventdate event probsoldivrrresult  \
0    LA|1    LA|628378  LA|430220  2020-10-08     6               None   
1    LA|2    LA|776032  LA|502531  2020-10-13     3               None   
2    LA|4    LA|775782  LA|463122  2020-10-13     4               None   
3    LA|6    LA|776289   LA|85158  2020-10-14     3               None   
4    LA|7    LA|776289   LA|85158  2020-10-14     6               None   
5    LA|8    LA|503904   LA|10763  2020-10-15     3               None   
6   LA|11    LA|671422  LA|395282  2020-08-17     4               None   
7   LA|12    LA|713205  LA|352088  2020-09-18     7               None   
8   LA|13    LA|756730  LA|452324  2020-10-19     6               None   
9   LA|14    LA|778187  LA|503387  2020-10-20     3               None   

  referralcasemanageafter locationcrisisorphhousing referralresult resultdate  \
0                    None                      None           None       None   
1                    No

/tmp/ipykernel_16836/4255036358.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


## Read data as dictionaries and JSON

In [3]:
from psycopg2.extras import RealDictCursor
import pandas as pd
import json
from datetime import date, datetime

# Custom JSON encoder to handle date and datetime fields
class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime, date)):
            return obj.isoformat()  # Convert to ISO 8601 format (string)
        return super().default(obj)
    
# Read data from the 'event' table as a dictionary
try:
    with connection.cursor(cursor_factory=RealDictCursor) as cursor:
        query = "SELECT * FROM event LIMIT 10;"  # Adjust the query as needed
        cursor.execute(query)
        
        rows = cursor.fetchall()
        
        # Convert rows to JSON format using custom encoder for date fields
        rows_json = json.dumps(rows, indent=4, cls=DateTimeEncoder)
        
        # Print rows as dictionaries
        for row in rows:
            print(row)
        
        # Optionally print the JSON format
        print("\nRows in JSON format:")
        print(rows_json)
        
except Exception as e:
    print(f"Error reading data from table: {e}")
finally:
    if connection:
        connection.close()

RealDictRow([('eventid', 'LA|1'), ('enrollmentid', 'LA|628378'), ('personalid', 'LA|430220'), ('eventdate', datetime.date(2020, 10, 8)), ('event', '6'), ('probsoldivrrresult', None), ('referralcasemanageafter', None), ('locationcrisisorphhousing', None), ('referralresult', None), ('resultdate', None), ('datecreated', datetime.datetime(2020, 10, 13, 11, 51, 17)), ('dateupdated', datetime.datetime(2020, 10, 13, 11, 51, 17)), ('userid', 9912), ('datedeleted', None), ('exportid', 430069)])
RealDictRow([('eventid', 'LA|2'), ('enrollmentid', 'LA|776032'), ('personalid', 'LA|502531'), ('eventdate', datetime.date(2020, 10, 13)), ('event', '3'), ('probsoldivrrresult', None), ('referralcasemanageafter', None), ('locationcrisisorphhousing', None), ('referralresult', None), ('resultdate', None), ('datecreated', datetime.datetime(2020, 10, 13, 15, 17, 14)), ('dateupdated', datetime.datetime(2020, 10, 13, 15, 17, 14)), ('userid', 3507), ('datedeleted', None), ('exportid', 430069)])
RealDictRow([('ev

In [4]:
if connection:
    connection.close()